In [84]:
import pandas as pd

In [96]:
qld = pd.read_excel("../Data/QLD FA Grants Data Tables.xlsx", sheet_name=None)

In [97]:
qld.keys()

dict_keys(['Grants', 'ERP', 'Revenue', 'Water', 'Value', 'Multipliers'])

In [98]:
grants = qld["Grants"]
erp = qld["ERP"]
revenue = qld["Revenue"]
water = qld["Water"]
value = qld["Value"]
multipliers = qld["Multipliers"]

In [99]:
for df in [grants, erp, revenue, water, value, multipliers]:
    df.set_index("Council", inplace=True)

In [100]:
qld = pd.concat([grants, erp, multipliers], axis=1)

In [101]:
qld.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77 entries, Aurukun Shire Council to Yarrabah Aboriginal Shire Council
Data columns (total 17 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   2024-25 GPG
entitlement          77 non-null     int64  
 1   Cash
adjustment
(for 2023-24)    77 non-null     int64  
 2   2024-25
Total cash
GPG           77 non-null     int64  
 3   2024-25 IRG
Entitlement          77 non-null     int64  
 4   Cash
adjustment
(for 2023-24)_1  77 non-null     int64  
 5   2024-25
Total cash
IRG           77 non-null     int64  
 6   2024-25
Cash FA
Grant            77 non-null     int64  
 7   Segment                          77 non-null     object 
 8   In year
population               77 non-null     int64  
 9   5-year
average
population        77 non-null     int64  
 10  Indigenous
population            77 non-null     int64  
 11  Segment uplift                   77 non-

In [102]:
qld.replace({
    "-": 0,
    ",": ""
}, regex=True, inplace=True)

In [103]:
for col in qld.columns:
    if col not in ["Council", "Segment"]:
        qld[col] = pd.to_numeric(qld[col], errors="coerce")

In [104]:
sub = qld.loc[qld["Segment"] == "Minimum Grant"]

In [105]:
sub.columns

Index(['2024-25 GPG\nentitlement', 'Cash\nadjustment\n(for 2023-24)',
       '2024-25\nTotal cash\nGPG', '2024-25 IRG\nEntitlement',
       'Cash\nadjustment\n(for 2023-24)_1', '2024-25\nTotal cash\nIRG',
       '2024-25\nCash FA\nGrant', 'Segment', 'In year\npopulation',
       '5-year\naverage\npopulation', 'Indigenous\npopulation',
       'Segment uplift', 'Weighted average ARIA', 'Indigenous %',
       'Advantage and disadvantage', 'Dispersion', 'Total multiplier'],
      dtype='object')

In [106]:
sub["2024-25 GPG\nentitlement"]/sub["In year\npopulation"]

Council
Brisbane City Council              25.598239
Bundaberg Regional Council         25.598237
Cairns Regional Council            25.598239
Fraser Coast Regional Council      25.598236
Gold Coast City Council            25.598239
Ipswich City Council               25.598237
Logan City Council                 25.598238
Mackay Regional Council            25.598241
Moreton Bay Regional Council       25.598239
Noosa Shire Council                25.598232
Redland City Council               25.598241
Rockhampton Regional Council       25.598235
Sunshine Coast Regional Council    25.598240
Toowoomba Regional Council         25.598237
Townsville City Council            25.598239
dtype: float64

In [108]:
minimum_gpg_per_capita = 25.5982
gpg_grant_pool = qld["2024-25 GPG\nentitlement"].sum()

qld["minimum grant"] = qld["In year\npopulation"] * minimum_gpg_per_capita
minimum_gpg_grant_pool = qld["minimum grant"].sum()

qld["allocation"] = qld["2024-25 GPG\nentitlement"] - qld["minimum grant"]

In [109]:
qld["scaled"] = qld["allocation"] / (gpg_grant_pool - minimum_gpg_grant_pool)